In [51]:
import numpy as np

# Define the grid world environment
GRID_ROWS, GRID_COLS = 4, 5  # 4x5 Grid World
actions = ['up', 'down', 'left', 'right']
action_space_size = len(actions)
action_symbols = ['U', 'D', 'L', 'R']  # For display purposes

In [52]:
import numpy as np

# Define the grid world environment
GRID_ROWS, GRID_COLS = 4, 5  # 4x5 Grid World
actions = ['up', 'down', 'left', 'right']
action_space_size = len(actions)
action_symbols = ['U', 'D', 'L', 'R']  # For display purposes

# Define the policy network parameters (random initialization)
theta = np.random.rand(GRID_ROWS, GRID_COLS, action_space_size)

# Define learning rate
alpha = 0.1  # Policy learning rate
gamma = 0.9  # Discount factor for future rewards

# Define the rewards grid
rewards = np.zeros((GRID_ROWS, GRID_COLS))
rewards[3, 4] = 1  # Goal position


# Function to print reward table
def print_reward_table():
    print("Reward Table:")
    print(rewards)
    print("\n")
print_reward_table()

Reward Table:
[[   0   -1   -1   -1   -1]
 [  -1   -1   -1   -1   -1]
 [  -1   -1 -100   -1   -1]
 [  -1   -1   -1   -1  100]]




In [53]:
# Policy softmax function to get action probabilities
def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)

# Function to choose action based on policy (probabilities from softmax)
def choose_action(state):
    row, col = state
    action_probs = softmax(theta[row, col])
    return np.random.choice(len(actions), p=action_probs)

# Function to take a step in the grid world
def take_step(state, action):
    row, col = state

    if action == 0:  # Up
        new_state = (max(0, row - 1), col)
    elif action == 1:  # Down
        new_state = (min(GRID_ROWS - 1, row + 1), col)
    elif action == 2:  # Left
        new_state = (row, max(0, col - 1))
    else:  # Right
        new_state = (row, min(GRID_COLS - 1, col + 1))

    reward = rewards[new_state]
    return new_state, reward

In [54]:
# Policy Gradient Update
def policy_gradient_update(state, action, G):
    row, col = state
    action_probs = softmax(theta[row, col])
    
    # Gradient ascent update (θ ← θ + α * G * ∇logπ)
    for a in range(action_space_size):
        grad_log_pi = (1 if a == action else 0) - action_probs[a]  # ∇logπ for each action
        theta[row, col, a] += alpha * G * grad_log_pi

# Function to display policy and probabilities side-by-side
def display_policy():
    direction_grid = np.full((GRID_ROWS, GRID_COLS), '', dtype='<U1')
    probability_grid = np.zeros((GRID_ROWS, GRID_COLS))

    for row in range(GRID_ROWS):
        for col in range(GRID_COLS):
            action_probs = softmax(theta[row, col])
            max_action = np.argmax(action_probs)
            direction_grid[row, col] = action_symbols[max_action]  # Direction with highest prob
            probability_grid[row, col] = np.max(action_probs)      # Highest probability

    # Display both grids side by side
    print("Policy (Direction) | Probabilities (Max)")
    for row in range(GRID_ROWS):
        direction_row = " ".join(direction_grid[row])
        probability_row = " ".join(f"{prob:.2f}" for prob in probability_grid[row])
        print(f"{direction_row}      | {probability_row}")

# Policy Gradient algorithm implementation
def policy_gradient(grid_iterations=100):
    for iteration in range(grid_iterations):
        state = (0, 0)  # Start state
        episode = []  # Store state, action, reward

        while state != (3, 4):  # Run until we reach the goal (bottom-right corner)
            action = choose_action(state)
            new_state, reward = take_step(state, action)
            episode.append((state, action, reward))
            state = new_state

        # Calculate the return G (sum of discounted rewards)
        G = 0
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = reward + gamma * G  # G ← r_t + γ * G

            # Update policy parameters based on the return G
            policy_gradient_update(state, action, G)

        # Every 10 iterations, print the policy and probabilities
        if (iteration + 1) % 10 == 0:
            print(f"Iteration {iteration + 1}:")
            display_policy()
            print("\n")

    # Final grid and path taken after training
    print("Final policy parameters (theta):")
    display_policy()

    # Show final path from start to goal
    state = (0, 0)
    path_taken = [state]
    while state != (3, 4):
        action = choose_action(state)
        state, _ = take_step(state, action)
        path_taken.append(state)

    print("Path taken:", path_taken)

# Call the function to run the policy gradient algorithm
policy_gradient(100)  # Run for 100 iterations
policy_gradient(100)  # Run for 100 iterations
policy_gradient(100)  # Run for 100 iterations
policy_gradient(100)  # Run for 100 iterations

Reward Table:
[[   0   -1   -1   -1   -1]
 [  -1   -1   -1   -1   -1]
 [  -1   -1 -100   -1   -1]
 [  -1   -1   -1   -1  100]]


Iteration 10:
Policy (Direction) | Probabilities (Max)
L R R R R      | 1.00 1.00 1.00 1.00 1.00
D R R D R      | 1.00 1.00 1.00 1.00 1.00
R D L D D      | 1.00 1.00 0.30 1.00 1.00
L L D R D      | 1.00 1.00 0.56 1.00 0.30


Iteration 20:
Policy (Direction) | Probabilities (Max)
L R R R R      | 1.00 1.00 1.00 1.00 1.00
D R R D R      | 1.00 1.00 1.00 1.00 1.00
R D L D D      | 1.00 1.00 0.30 1.00 1.00
L L D R D      | 1.00 1.00 1.00 1.00 0.30


Iteration 30:
Policy (Direction) | Probabilities (Max)
L R R R R      | 1.00 1.00 1.00 1.00 1.00
D R R D R      | 1.00 1.00 1.00 1.00 0.93
R D L D D      | 1.00 1.00 0.30 1.00 1.00
L L D R D      | 1.00 1.00 1.00 1.00 0.30


Iteration 40:
Policy (Direction) | Probabilities (Max)
L R R R R      | 1.00 1.00 1.00 1.00 1.00
D R R D R      | 1.00 1.00 1.00 1.00 1.00
R D L D D      | 1.00 1.00 0.30 1.00 1.00
L L D R D      

## Key Aspects:

### 1. Softmax Function:
The policy is derived by converting policy parameters `theta` to action probabilities using the softmax function:

\[
\pi(a|s; \theta) = \frac{e^{\theta_{s,a}}}{\sum_{b} e^{\theta_{s,b}}}
\]

### 2. Action Selection:
For each state `(row, col)`, the agent chooses an action based on the probabilities from the softmax function.

### 3. Policy Update:
The policy is updated using the gradient ascent rule for policy gradient:

\[
\theta_{s,a} \leftarrow \theta_{s,a} + \alpha \cdot G \cdot \nabla \log \pi(a|s; \theta)
\]

### 4. Return Calculation (G):
The return `G` is computed as the discounted sum of future rewards:

\[
G_t = r_t + \gamma G_{t+1}
\]

### 5. Grid and Path Printing:
The grid updates are printed every 10 steps to show progress. The final path taken by the agent after training is also printed.
